# 1. Objet

Ce notebook a pour objet de traiter les fichiers bruts mis en entrée dans le répertoire `data`. Il s'agit des fichiers : 
- d'historiques de vente
- de référentiel client

Etant donné la simplicité des traitements faits sur le référentiel client, ils sont fait directement dans ce fichier.

# 2. Setup et imports

In [1]:
from pathlib import Path
import sys
import datetime
import pandas as pd
idx = pd.IndexSlice
project_root = str(Path(sys.path[0]).parents[0].absolute())
project_root
if project_root not in sys.path:
    sys.path.append(project_root)
data_path = Path('..') / 'data'
persist_path = Path('..') / 'persist'
from IPython.display import display, HTML
display(HTML("<style>.container { width:90%; }</style>"))

Définition du client Dask

In [2]:
from dask.distributed import Client
import dask.dataframe as dd
# SI BESOIN DE RETRAITER LES CSV, UTILISER LA LIGNE CI-DESSOUS !
# client = Client(n_workers=1, threads_per_worker=8, processes=True, memory_limit='31GB') # n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB'
client = Client()
client

/var/lib/anaconda3/envs/py38/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44105 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:46694 Dashboard: http://127.0.0.1:44105/status,Cluster Workers: 4 Cores: 8 Memory: 33.56 GB


2. Définition de la liste des succursales

In [3]:
orgacom_list = [
    '1ALO',
    '1BFC',
    '1CAP',
    '1CTR',
    '1EXP',
    '1LRO',
    '1LXF',
    '1NCH',
    '1OUE',
    '1PAC',
    '1PNO',
    '1PSU',
    '1RAA',
    '1SOU',
    '2BRE',
    '2CAE',
    '2CTR',
    '2EST',
    '2IDF',
    '2IFC',
    '2MPY',
    '2NOR',
    '2RAA',
    '2SES',
    '2SOU',
]

# 3. Données pour contrôle

On récupère des données agrégées issues directement de la LISTCUBE dans PBI, afin de contrôler à la volée que l'extraction et le chargement des données à une maille fine se sont bien passés.

1. Définition des champs qui serviront à faire le contrôle

In [4]:
fields_to_compare = ['weight', 'brutrevenue', 'margin']

2. Définition des mois à contrôler

In [5]:
months = [
    '201707',
    '201708',
    '201709',
    '201710',
    '201711',
    '201712',
    '201801',
    '201802',
    '201803',
    '201804',
    '201805',
    '201806',
    '201807',
    '201808',
    '201809',
    '201810',
    '201811',
    '201812',
    '201901',
    '201902',
    '201903',
    '201904',
    '201905',
    '201906',
    '201907',
    '201908',
    '201909',
    '201910',
    '201911',
    '201912',
    '202001',
    '202002',
    '202003',
    '202004',
    '202005',
    '202006',
    '202007',
    '202008',
    '202009',
    '202010',
    '202011',
    '202012',
    '202101',
]

3. Chargement des données

In [6]:
ctrle = pd.read_csv(
    data_path / 'detrompeur_data.csv',
    encoding='latin1',
    sep=';',
    names=['orgacom', 'month', 'year', 'currency', 'weight_unit', 'weight_unit2', 'weight', 'weight2', 'brutrevenue', 'netrevenue', 'net2revenue', 'margin'],
    skiprows=[0],
    dtype={
        'orgacom': pd.CategoricalDtype(orgacom_list),
        'month': pd.CategoricalDtype(months),
        'year': 'object',
        'weight': 'float',
    },
    decimal=',',
    thousands=' ',
).loc[lambda x: x.month.isin(months)].set_index(['orgacom', 'month']).sort_index()
ctrle

year currency weight_unit  weight_unit2       weight  weight2  \
orgacom month                                                                   
1ALO    201707  2017      EUR          KG           NaN  3144714.984      0.0   
        201708  2017      EUR          KG           NaN  3329429.530      0.0   
        201709  2017      EUR          KG           NaN  4543058.400      0.0   
        201710  2017      EUR          KG           NaN  4406942.653      0.0   
        201711  2017      EUR          KG           NaN  4670216.793      0.0   
...              ...      ...         ...           ...          ...      ...   
2SOU    202009  2020      EUR          KG           NaN  2019045.566      0.0   
        202010  2020      EUR          KG           NaN  1729008.134      0.0   
        202011  2020      EUR          KG           NaN  1552341.359      0.0   
        202012  2020      EUR          KG           NaN  1508480.846      0.0   
        202101  2021      EUR          KG           NaN  1559127.769      0.0   

                brutrevenue   netrevenue  net2revenue      margin  
orgacom month                                                      
1ALO    201707  10426960.20  10024879.18   9742979.65  1666902.18  
        201708  10984996.41  10579208.70  10294519.29  1817023.14  
        201709  14679332.61  14122353.46  13743135.09  2374842.11  
        201710  14542930.00  13948005.52  13567019.14  2298395.93  
        201711  15618497.28  15023089.41  14630158.70  2468127.12  
...                     ...          ...          ...         ...  
2SOU    202009   3888583.36   3694176.69   3558897.59   817271.63  
        202010   3352393.38   3187072.99   3065034.48   697716.19  
        202011   2850131.34   2705915.02   2590223.99   523473.96  
        202012   2728676.03   2594373.68   2484134.90   487176.95  
        202101   2858547.33   2718724.05   2609435.75   549102.26  

[1052 rows x 10 columns]

# 4. Historiques de vente


Constitution d'un DataFrame par Succursale

Dans cette partie, on va constituer un DataFrame par succursale qu'on va persister sur le disque.

1. Définition du format du csv

In [7]:
init_fields = {'orgacom': pd.CategoricalDtype(orgacom_list),
          'month': pd.CategoricalDtype(months),
          'week': 'category',
          'date': 'object',
          'pricetype_init': 'object',
          'pricetype_applied': 'object',
          'mercu_init': 'object', 
          'mercu_applied': 'object',
          'client': 'object',
          'doctype': 'object',
          'origin': pd.CategoricalDtype(['TV', 'VR', 'EDI', 'WEB', '#', 'SCHR', 'TELE',
                                         'MUEN', 'FRN', 'DFUE'], ordered=True),
          'salesgroup': 'object',
          'material': 'object',
          'brutrevenue': 'float',
          'brutrevcur': 'object', 
          'netrevenue': 'float', 
          'netrevcur': 'object',
          'weight': 'float',
          'weightunit': 'object',
          'margin': 'float', 
          'margincur': 'object', 
          'marginperkg': 'float',
         }

2. Paramètres pour l'interprétation du csv

In [8]:
read_csv_kwargs = dict(
    sep=";",
    header=None,
    names=list(init_fields.keys()),      
    dtype=init_fields, 
    parse_dates=['date'],    
)

3. Définition des fichiers bruts

Ces fichiers doivent être dans le répertoire `data`.

In [9]:
raw_data_filenames = [
    'export_total.csv',  # premier export, qui a foiré sur l'année 2020 en gros
    'export_complementaire.csv',  # première moitié de 2020 - interrompu ensuite
    'export_complementaire_3.csv',  # export de la seconde moitié de 2020
    'export_total_16_17_dec_2019.csv',  # 16 et 17 décembre 2019, qui manquaient
    'export_total_29-01_09_2020.csv',
    'export_total_22-24_09_2020.csv',
    'export_total_14-17_09_2020.csv',
    'export_total_10-13_09_2020.csv',
    'export_total_06-09_09_2020.csv',
    'export_total_02-05_09_2020.csv',
    'export_total_23-08-2020_2PES.csv',
    'export_total_23-08-2020_PPF.csv',
    'export_total_24-26_08_2020.csv',
    'export_total_27-08-2020.csv',
    'export_total_28_08_2020.csv',
    'export_total_18-19_09_2020.csv',
    'export_total_20-09-2020_1PPF.csv',
    'export_total_20-09-2020_2PES.csv',
    'export_total_21-09-2020.csv',
#     'EXTRACT_LIGNES_VENTES_V3.csv',  # NE PAS PRENDRE ! Juste pour vérifier que mon outil de contrôle identifiait bien les écarts liés à l'absence de soum (spoiler alert: c'est bon!)
]
raw_data_paths = [data_path / filename for filename in raw_data_filenames]

4. Lecture des csv, et enregistrement sur le disque

In [10]:
ddf = dd.read_csv(
    raw_data_paths,
    **read_csv_kwargs,
).set_index('date')

# ATTENTION ! La ligne suivante est très coûteuse et nécessite d'avoir 1 worker avec toute la RAM du serveur!
# Cf. la ligne de création du client en entête de ce notebook
ddf.to_parquet(persist_path / 'raw_dataxxxxxx.parquet')

5. Contrôle de la conformité des données

In [11]:
ddf = dd.read_parquet(persist_path / 'raw_dataxxxxxx.parquet')

In [12]:
grouped = ddf.loc[:, ['orgacom', 'month', *fields_to_compare]].groupby(['orgacom', 'month']).sum().compute()
grouped

weight  brutrevenue      margin
orgacom month                                       
1ALO    201707  3144714.984  10426960.20  1666902.18
        201708  3329429.530  10984996.41  1817023.14
        201709  4543058.400  14679332.61  2374842.11
        201710  4406942.653  14542930.00  2298395.93
        201711  4670216.793  15618497.28  2468255.18
...                     ...          ...         ...
2SOU    202009  2019045.566   3888583.36   817271.63
        202010  1729008.134   3352393.38   697716.19
        202011  1552341.359   2850131.34   523473.96
        202012  1508480.846   2728676.03   487176.95
        202101  1559127.769   2858547.33   549102.26

[1075 rows x 3 columns]

In [13]:
delta = (
    grouped
    .merge(
        ctrle.loc[:, fields_to_compare],
        suffixes=(None, '_ctrle'),
        left_index=True,
        right_index=True,
        indicator=False,
        how='outer'
    )
    .fillna(0)
    .assign(
        **{indicator + '_delta': lambda x, i=indicator: (x[i] - x[i + '_ctrle']) / (x[i + '_ctrle'])
           for indicator in fields_to_compare
          }
    )
    .drop([*[indicator for indicator in fields_to_compare], *[indicator + '_ctrle' for indicator in fields_to_compare]], axis=1)
    .unstack('orgacom')
    .swaplevel(axis=1)
    .sort_index(axis=1)
)
with pd.option_context('display.max_columns', None, 'display.float_format', lambda x: f'{x:.2%}'):
    display(delta.style.bar(align='mid', axis=None))

In [14]:
delta.to_pickle(persist_path / 'current_delta.pkl')

distributed.nanny - WARNING - Restarting worker


6. Itération - ANCIENNE VERSION, PLUS NECESSAIRE.

In [ ]:
# delta_list = []

# for cpt0, orgacom in enumerate(orgacom_list):
#     print(f'---------------------------------------------------')
#     print(f'--------       Succursale {orgacom} - {cpt0 + 1}/{len(orgacom_list)}       --------')
#     print(f'---------------------------------------------------\n')
#     df_list = []
#     for cpt, raw_data_filename in enumerate(raw_data_filenames):
#         print(f'Traitement du fichier {cpt + 1}/{len(raw_data_filenames)} - {raw_data_filename}')
#         print(f'          --------------------           ')
#         iterator = pd.read_csv(data_path / raw_data_filename, **iterator_kwargs)
#         print(f"{datetime.datetime.now()} - Début de l'itération")
#         for cpt2, content in enumerate(iterator):
#             print(f'{datetime.datetime.now()} - Run {cpt2 + 1}')
#             df_list.append(content.loc[lambda x: x.orgacom == orgacom])
#     print(f'{datetime.datetime.now()} - Fin du traitement des fichiers pour la succursale {orgacom} !')
#     print(f'          --------------------           ')    
#     print(f'{datetime.datetime.now()} - Concaténation pour la succursale {orgacom}')
#     oc_df = pd.concat(df_list, axis=0)
#     oc_df = oc_df.reset_index(drop=True)

#     print(f'{datetime.datetime.now()} - Enregistrement sur disque pour la succursale {orgacom}\n')
#     filename = f'data_{orgacom}.pkl'    
#     oc_df.to_pickle(persist_path / 'rawbyoc' / filename)
    
#     print(f'{datetime.datetime.now()} - Contrôle de la conformité des données pour la succursale {orgacom}')
#     reference = ctrle.loc[idx[orgacom, months], fields_to_compare]
#     to_check = oc_df.groupby(['orgacom', 'month'], observed=True)[fields_to_compare].sum()
#     aligned = reference.align(to_check, fill_value=0.)
#     delta_list.append((aligned[0] - aligned[1]) / aligned[0])

#     del(oc_df)

# print('Traitement terminé !')
    
# delta = pd.concat(delta_list).unstack('orgacom').swaplevel(axis=1).sort_index(axis=1)
# del(delta_list)
# with pd.option_context('display.float_format', lambda x: f'{x:.2%}', 'display.max_columns', None):
#     display(delta)
# delta.to_pickle(persist_path / 'current_delta.pkl')


# 4. Référentiel client

1. Définition des entêtes des fichiers à intégrer

In [ ]:
field_names = [
    'client',
    'V',
    'groupecompte',
    'nom',
    'postalcode',
    'seg1',
    'seg2',
    'seg3',
    'seg4',
    'cat',
    'sscat',
    'saiso',
    'surcat',
    'ecom',
    'sectact',
    'canal',
    'orgacom',
    'grpclt1',
    'grpclt2',
    'grpclt3',
    'grpclt4',
    'grpclt5',
    'agence',
    'condexp',
    'pricetype',
    'relationtype',
    'pilcom',
    'hier4',
    'hier3',
    'hier2',
    'hier1',
    'adrnr',
    'city',
    'SIRET',
    'hier4_l',
    'SIREN',
    'street',
    'V2',
    'hier3_l',
    'hier2_l',
    'hier1_l',
    'mandant_deletion_indicator',
    'sales_org_deletion_indicator',    
]

2. Définition des formats initiaux (i.e. l'interprétation "brute" du fichier)

In [ ]:
initial_dtypes = {
    'client': 'object',
    'V': 'object',
    'groupecompte': 'object',
    'nom': 'object',
    'postalcode': 'object',
    'seg1': 'object',
    'seg2': 'object',
    'seg3': 'object',
    'seg4': 'object',
    'cat': 'object',
    'sscat': 'object',
    #'saiso': 'bool', # défini dans le converter du read_csv
    'surcat': 'object',
    #'ecom': 'bool', # défini dans le converter du read_csv
    'sectact': 'object',
    'canal': 'object', 
    'orgacom': 'object', 
    'grpclt1': 'object',
    'grpclt2': 'object',
    'grpclt3': 'object',
    'grpclt4': 'object',
    'grpclt5': 'object',
    'agence': 'object', 
    'condexp': 'object', 
    'pricetype': 'object', 
    'relationtype': 'object',
    'pilcom': 'object',
    'hier4': 'object',
    'hier3': 'object',
    'hier2': 'object',
    'hier1': 'object',
    'adrnr': 'object',
    'city': 'object',
    'SIRET': 'object',
    'hier4_l': 'object',
    'SIREN': 'object',
    'street': 'object',
    'V2': 'object',
    'hier3_l': 'object',
    'hier2_l': 'object',
    'hier1_l': 'object',
#     'mandant_deletion_indicator': 'bool',  # défini dans le converter du read_csv
#     'sales_org_deletion_indicator': 'bool',  # défini dans le converter du read_csv 
}

3. Définition des formats "cibles" du DataFrame. On fait en 2 temps pour la gestion des données `Categorical` (on concatène d'abord les dataframes pour que la converions en `Categorical` "voie" l'ensemble des valeurs).

In [ ]:
target_dtypes = {
    'client': 'object',
    'V': 'category',
    'groupecompte': 'category',
    'nom': 'object',
    'postalcode': 'category',
    'seg1': 'category',
    'seg2': 'category',
    'seg3': 'category',
    'seg4': 'category',
    'cat': 'category',
    'sscat': 'category',
    #              'saiso': 'bool',
    'surcat': 'category',
    #              'ecom': 'bool', 
    'sectact': 'category',
    'canal': 'category', 
    'orgacom': 'category', 
    'grpclt1': 'category',
    'grpclt2': 'category',
    'grpclt3': 'category',
    'grpclt4': 'category',
    'grpclt5': 'category',
    'agence': 'category', 
    'condexp': 'category', 
    'pricetype': 'category', 
    'relationtype': 'category',
    'pilcom': 'category',
    'hier4': 'object',
    'hier3': 'object',
    'hier2': 'object',
    'hier1': 'object',
    'adrnr': 'object',
    'city': 'object',
    'SIRET': 'object',
    'hier4_l': 'object',
    'SIREN': 'object',
    'street': 'object',
    'V2': 'object',
    'hier3_l': 'object',
    'hier2_l': 'object',
    'hier1_l': 'object',
#     'mandant_deletion_indicator': 'bool',  # défini dans le converter du read_csv
#     'sales_org_deletion_indicator': 'bool',  # défini dans le converter du read_csv     
            }

4. Définition des arguments du read_csv

In [ ]:
csv_kwargs = dict(
    sep=';',
    encoding='latin1',
    skiprows=[0], # skip header row
    names=field_names,
    dtype=initial_dtypes,
    converters={
        'saiso': lambda x: True if x == 'YES' else False, 
        'ecom': lambda x: True if x == 'X' else False,
        'mandant_deletion_indicator': lambda x: True if x == 'X' else False,
        'sales_org_deletion_indicator': lambda x: True if x == 'X' else False,
    },
)

5. Lecture, concaténation des dataframes, et définition des catégories

In [ ]:
filenames = [
    'ref_clt_1PPF.csv',
    'ref_clt_2PES.csv',
]
df_list = [
    pd.read_csv(
        data_path / 'clt' / filename,
        **csv_kwargs,
    ) for filename in filenames
]
df_clt = pd.concat(df_list, axis=0)
df_clt = df_clt.astype(target_dtypes)

6. Nettoyages divers

In [ ]:
# remove emppty client codes
df_clt = df_clt.loc[~df_clt.client.isna()]

# zero pad numeric clients codes
num_clt_mask = df_clt['client'].str.isnumeric()
df_clt.loc[num_clt_mask, 'client'] = df_clt.loc[num_clt_mask, 'client'].str.zfill(10)

#remove duplicated clients
print(f'Clients count before cleansing : {df_clt.client.count()}')
print(f"Duplicated clients before cleansing: {sum(df_clt.loc[:, ['client', 'orgacom']].duplicated(keep=False))}")
df_clt = df_clt.loc[~(df_clt.loc[:, ['client', 'orgacom']].duplicated(keep=False) & df_clt.sectact.isna())]
print(f"Clients count after cleansing : {df_clt.client.count()}")
print(f"Duplicated clients after cleansing: {sum(df_clt.loc[:, ['client', 'orgacom']].duplicated(keep=False))}")

# set the index
df_clt.set_index(['orgacom', 'client'], inplace=True, drop=True)

In [ ]:
if sum(df_clt.index.duplicated()):
    raise RuntimeError('Something went wrong on the client dataframe! Some indices are duplicated!')

7. Persistage sur le disque

In [ ]:
df_clt.to_pickle(persist_path / 'clt.pkl')

In [24]:
ddf = dd.read_parquet(Path('..') / 'persist' / 'raw_data.parquet')
janfev2019 = ddf.loc[(ddf.index >= '2020-01-01') & (ddf.index <= '2020-02-29')].compute()

In [10]:
df_clt = pd.read_pickle(persist_path / 'clt.pkl')

In [12]:
len(janfev2019)

11071436

In [36]:
del(janfev2019)

In [25]:
janfev2019 = janfev2019.merge(df_clt, on=['orgacom', 'client'], how='left')

In [26]:
janfev2019['branch'] = janfev2019['orgacom'].str[0]

In [27]:
maskRHD = (janfev2019.seg1 == 'Z3')

In [ ]:
periods = {
    'janfev2019': ('2019-01-01', '2019-02-28'),
    'janfev2020': ('2020-01-01', '2020-02-29'),
}

In [22]:
toto =list()

In [28]:
toto.append(
    janfev2019
    .loc[maskRHD]
    .groupby(['branch', 'seg3'], observed=True, as_index=False)
    [['margin', 'weight', 'brutrevenue']]
    .sum()
    .assign(marginperkg=lambda x: x['margin'] / x['weight'])
)

In [30]:
result = pd.concat(toto, axis=0, keys=['2019', '2020'])
result

branch seg3        margin        weight   brutrevenue  marginperkg
2019 0      1   ZK  1.266626e+07  7.977041e+06  4.383251e+07     1.587840
     1      1   ZI  1.392282e+07  2.441042e+07  7.311552e+07     0.570364
     2      1   ZJ  1.326202e+07  4.284610e+07  1.356818e+08     0.309527
     3      1   ZL  3.699327e+06  6.968475e+06  2.856577e+07     0.530866
     4      2   ZK  5.084667e+06  4.481158e+06  1.337732e+07     1.134677
     5      2   ZI  7.352117e+06  1.676207e+07  2.835760e+07     0.438616
     6      2   ZJ  4.059258e+06  1.838051e+07  4.012660e+07     0.220846
     7      2   ZL  1.965326e+06  5.050939e+06  1.121156e+07     0.389101
2020 0      1   ZK  1.395687e+07  8.537419e+06  4.855055e+07     1.634788
     1      1   ZL  3.614583e+06  7.241145e+06  2.923861e+07     0.499173
     2      1   ZI  1.494918e+07  2.309917e+07  7.282340e+07     0.647174
     3      1   ZJ  1.322394e+07  4.235137e+07  1.395299e+08     0.312243
     4      2   ZK  5.697039e+06  4.794022e+06  1.471496e+07     1.188363
     5      2   ZL  1.876259e+06  4.503466e+06  1.059573e+07     0.416626
     6      2   ZI  7.565415e+06  1.664098e+07  2.827853e+07     0.454625
     7      2   ZJ  4.220504e+06  1.860024e+07  4.095801e+07     0.226906

In [35]:
result.reset_index().rename({'level_0': 'year'}, axis=1).set_index(['branch', 'year']).loc[lambda x: x.seg3 == 'ZI'].sort_index()

level_1 seg3        margin        weight   brutrevenue  \
branch year                                                           
1      2019        1   ZI  1.392282e+07  2.441042e+07  7.311552e+07   
       2020        2   ZI  1.494918e+07  2.309917e+07  7.282340e+07   
2      2019        5   ZI  7.352117e+06  1.676207e+07  2.835760e+07   
       2020        6   ZI  7.565415e+06  1.664098e+07  2.827853e+07   

             marginperkg  
branch year               
1      2019     0.570364  
       2020     0.647174  
2      2019     0.438616  
       2020     0.454625